# Window Functions, Part 2: Ranking


In this part, I'll teach you how to use window functions for something very useful – creating rankings. 

 - Which x is the best? 

 - Which x is in 3rd place? 

 - Which is the worst? 

These are the questions we're going to answer in this part.

Today, we're going to deal with a development studio that produces games for mobile platforms. 

Let's take a look at it.


Go to the fiddle, here: https://www.db-fiddle.com/f/fWMdFHYRpTy8FBfKq4roUA/1



# Exercise

Select all the information from table game.

Each game has an id and a name and is runnable on a specific platform. It is characterized by a specific genre, has an editor_rating and size in MB. 

The column released specifies when the game was released, whereas updated – when the game was last updated.


Each purchase of a game is stored in this table. The purchase has its own id, the id of the game, the price paid and the date of purchase.

# Exercise

Select all the information from the purchase table.

Each purchase of a game is stored in this table. The purchase has its own id, the id of the game, the price paid and the date of purchase.


# Ranking functions - introduction

Let's get down to work. So far, you've learned how to use window functions with aggregate functions that you already know – SUM(), COUNT(), AVG(), MAX() and MIN().

Now, we'll teach you different functions that go well with OVER() – ranking functions. The general syntax is as follows:

`<ranking function> OVER (ORDER BY <order by columns>)` 


We'll introduce a few possible ranking functions in the next exercises.

As to OVER (ORDER BY col1, col2...), this is the part where you specify the order in which rows should be sorted and therefore ranked.


# Function RANK()

Alright. We'll start with the most widely used ranking function: RANK(). The syntax is as follows:

`RANK() OVER (ORDER BY ...)`


What does RANK() do? 

It returns the rank (a number) of each row with respect to the sorting specified within parentheses.

ORDER BY sorts rows and shows them in a specific order to you. 

RANK() OVER(ORDER BY ...) is a function that shows the rank(place, position) of each row in a separate column.

Let's look at an example from our database:

```
SELECT
  name,
  platform,
  editor_rating,
  RANK() OVER(ORDER BY editor_rating)
FROM game;
```



The first three columns are quite obvious, but look what happens next: we want to return the rank (RANK()) of each row when we sort them by the column editor_rating (OVER(ORDER BY editor_rating).

Look at the result of the example query.

As you can see, we get the rank of each game in the last column. There are 3 games with the lowest score – 4. All of them got rank 1.

The next game, with score 5, got rank 4, not 2. 

That's how RANK() works. 


There were three games before the game with score 5, so, being the 4th game, it got rank 4 – regardless of the fact that the other three all got rank 1. 

RANK() will always leave gaps in numbering when more than 1 row share the same value.

# Exercise

For each game, show name, genre, date of update and its rank. The rank should be created with RANK() and take into account the date of update.

```
SELECT
  name,
  genre,
  updated,
  RANK() OVER(ORDER BY updated)
FROM game;
```

# DENSE_RANK()

As we said, RANK() will always leave gaps in numbering when more than 1 row share the same value. You can change that behavior by using another function: DENSE_RANK():

```
SELECT
  name,
  platform,
  editor_rating,
  DENSE_RANK() OVER(ORDER BY editor_rating)
FROM game;
```


DENSE_RANK gives a 'dense' rank indeed, i.e. there are no gaps in numbering.

Look at the output from the query above. Right now, go loook at it.


We've now got three rows with rank 1, followed by a fourth row with rank 2. That's the difference between RANK() and DENSE_RANK() – the latter never leaves gaps.


# Exercise

Use DENSE_RANK() and for each game, show name, size and the rank in terms of its size.



```
SELECT
  name,
  size,
  DENSE_RANK() OVER(ORDER BY size)
FROM game;
```

# ROW_NUMBER()

All right, and now, let us take a look at yet another way of ranking:

```
SELECT
  name,
  platform,
  editor_rating,
  ROW_NUMBER() OVER(ORDER BY editor_rating)
FROM game;
```

Let's run the example to see what's different now.



# Exercise

Run the example above and see the difference in ranking.

Now, each row gets its own, unique rank number, so even rows with the same value get consecutive numbers.

```
SELECT
  name,
  platform,
  editor_rating,
  ROW_NUMBER() OVER(ORDER BY editor_rating)
FROM game;
```

# ROW_NUMBER() – practice

As you can see, ROW_NUMBER() gives a unique rank number to each row. Even those rows which share the same editor_rating value got different ranks that are expressed as consecutive numbers.

The only problem is with the order of these consecutive numbers. 

You could ask yourself – how does my database determine which of the games with editor_rating = 4 gets 1, 2 or 3 as the rank? 

The answer is – it doesn't, really. 

The order is nondeterministic. When you execute ROW_NUMBER(), you never really know what the output will be.

Now, how about an exercise for you?

# Exercise

Use ROW_NUMBER() and for each game, show their name, date of release and the rank based on the date of release.



```
SELECT
  name,
  released,
  ROW_NUMBER() OVER(ORDER BY released)
FROM game;
```

# Exercise

For each game, show its name, genre and date of release. In the next three columns, show RANK(), DENSE_RANK() and ROW_NUMBER() sorted by the date of release.



```
SELECT
  name,
  genre,
  released,
  RANK() OVER(ORDER BY released),
  DENSE_RANK() OVER(ORDER BY released),
  ROW_NUMBER() OVER(ORDER BY released)
FROM game;
```

# RANK() OVER(ORDER BY ... DESC)

You may have noticed that the rank given by the function may be counter-intuitive to some extent. When we showed the rank sorted by editor_rating, the game with the lowest score got a rank of 1. We usually want it the other way around – the first place should be occupied by the best game. Luckily, this requires a very minor change to our query:


```
SELECT
  name,
  platform,
  editor_rating,
  RANK() OVER(ORDER BY editor_rating DESC)
FROM game;
```

Voila, add DESC after the column name in OVER( ... ) and the ranking order is reversed.



# Exercise

Let's use `DENSE_RANK()` to show the latest games from our studio. For each game, show its name, genre, date of release and DENSE_RANK() in the descending order.

```
SELECT
  name,
  genre,
  released,
  DENSE_RANK() OVER(ORDER BY released DESC)
FROM game;
```

# RANK() with ORDER BY many columns

Yet another thing you can do is rank by multiple columns, each of them in the ascending or descending order of your choice. Let's pretend that a player named John has limited space on his phone, but he wants to install a relatively recent game. Take a look:

```
SELECT
  name,
  genre,
  editor_rating,
  RANK() OVER(ORDER BY released DESC, size ASC)
FROM game;
```

We first sort the rows by the release date, with the latest games coming first, and then by size, with the smallest games appearing before bigger games.

# Exercise

We want to find games which were both recently released and recently updated. For each game, show name, date of release and last update date, as well as their rank: use ROW_NUMBER(), sort by release date and then by update date, both in the descending order.




```
SELECT
  name,
  released,
  updated,
  ROW_NUMBER() OVER(ORDER BY released DESC, updated DESC)
FROM game;
```

# Ranking and ORDER BY

Very good! You may wonder if you can use regular ORDER BY with the ranking functions. Of course, you can. The ranking function and the external ORDER BY are independent. The ranking function returns the rank with the respect to the order provided within OVER. Let's look at the example:

````
SELECT
  name,
  RANK() OVER (ORDER BY editor_rating)
FROM game
ORDER BY size DESC;
````

The query returns the name of the game and the rank of the game with respect to editor ranking. The returned rows are ordered by size of the game in descending way.



# Exercise

For each game find its name, genre, its rank by size. Order the games by date of release with newest games coming first.

```
SELECT
  name,
  genre,
  RANK() OVER(ORDER BY size)
FROM game
ORDER BY released DESC;
```

# Exercise

For each purchase, find the name of the game, the price, and the date of the purchase. Give purchases consecutive numbers by date when the purchase happened, so that the latest purchase gets number 1. Order the result by editor's rating of the game.


```
SELECT
  name,
  price,
  date,
  ROW_NUMBER() OVER(ORDER BY date DESC)
FROM purchase, game
WHERE game.id = game_id
ORDER BY editor_rating;
```

# NTILE(X)

The last function we'll introduce in this section is NTILE(X). It distributes the rows into a specific number of groups, provided as X. For instance:

```
SELECT
  name,
  genre,
  editor_rating,
  NTILE(3) OVER (ORDER BY editor_rating DESC)
FROM game;
```

In the above example, we create three groups with NTILE(3) that are divided based on the values in the column editor_rating. The "best" games will be put in group 1, "average" games in group 2, "worst" games in group 3. See the picture below:




<img src = 'https://learnsql.com/static/postgresql-window-functions-window-functions-part4-ex15.png'>

Note that if the number of rows is not divisible by the number of groups, some groups will have one more element than other groups, with larger groups coming first.

# Exercise

We want to divide games into 4 groups with regard to their size, with biggest games coming first. For each game, show its name, genre, size and the group it belongs to.

```
SELECT
  name,
  genre,
  size,
  NTILE(4) OVER (ORDER BY size DESC)
FROM game;
```

# Exercise

Split the games into 5 groups based on their date of last update. The most recently updated games should come first. For each of them, show the name, genre, date of update and the group they were assigned to. In the result, notice how many items the groups have (varying value).

```
SELECT
  name,
  genre,
  updated,
  NTILE(5) OVER(ORDER BY updated DESC)
FROM game;
```

In the previous section, we've introduced ranking functions whose result was shown as an additional column in our query results. 

Our game table is pretty small, so it's easy to identify the first, second, or third place manually. In real life, however, we deal with huge tables and looking for one particular rank can be troublesome.

In this section, we will learn how to create queries that, for instance, return only the row with rank 1, 5, 10, etc. 

This cannot be accomplished with a simple query – we will need to create a complex one. 

For this purpose, we'll use Common Table Expressions. 

An example may look like this:

```
WITH ranking AS (
  SELECT
    name,
    RANK() OVER(ORDER BY editor_rating DESC) AS rank
  FROM game
)

SELECT name
FROM ranking
WHERE rank = 2;
```


The query returns the name of the game which gets rank number 2 with respect to editor rating. Don't worry if the query looks complicated. We'll explain it in a second.


# Create the ranking

Ok, we want to answer the following question: what is the name of the game with rank 2 in terms of best editor_rating? 

We create the SQL query to answer this problem in two steps.

In step 1, we create a ranking, just as we did in the previous section:

```
SELECT name,
  RANK() OVER(ORDER BY editor_rating DESC) AS rank
FROM game;
```

This step should be quite obvious now. Let's run it to confirm that it works.

## The full query

Alright, the query worked the way we wanted.

Now, the second step is to treat our previous example as a subquery and put it in the FROM clause. As you can remember, we previously wrote:

```
SELECT
  name,
  RANK() OVER(ORDER BY editor_rating DESC) AS rank
FROM game;
and now we'll write this:

WITH ranking AS (
  SELECT
    name,
    RANK() OVER(ORDER BY editor_rating DESC) AS rank
  FROM game
)

SELECT name
FROM ranking
WHERE rank = 2;
```

The first line (WITH ranking AS) tells that what follows is called ranking. 

Inside the parentheses, we provide the query which we created in the previous step. In the end, all we do is select the row(s) with rank = 2 from the query we named ranking.

It's time for some practice, let's give it a try.

# Exercise

Find the name, genre and size of the smallest game in our studio.

Remember the steps:

1. Create the ranking query so that the smallest game gets rank 1.

2. Use WITH to select rows with rank 1.


```

WITH ranking AS (
  SELECT
    name,
    genre,
    size,
    RANK() OVER(ORDER BY size) AS rank
  FROM game
)

SELECT
  name,
  genre,
  size
FROM ranking
WHERE rank = 1;
```


# Exercise

Show the name, platform and update date of the second most recently updated game.



```
WITH ranking AS (
  SELECT
    name,
    platform,
    updated,
    RANK() OVER(ORDER BY updated DESC) AS rank
  FROM game
)

SELECT
  name,
  platform,
  updated
FROM ranking
WHERE rank = 2;
```

# Summary

Okay, let's review what we've learned in this part:

- The most basic usage of ranking functions is: RANK() OVER(ORDER BY column1, column2...).

- The ranking functions we have learned:

- RANK() – returns the rank (a number) of each row with respect to the sorting specified within parentheses.

- DENSE_RANK() – returns a 'dense' rank, i.e. there are no gaps in numbering.

- ROW_NUMBER() – returns a unique rank number, so even rows with the same value get consecutive numbers.

- NTILE(x) – distributes the rows into a specific number of groups, provided as x.

To get col1 of the row with rank place1 in a ranking sorted by col2, write:

```
WITH ranking AS
  (SELECT
    RANK() OVER (ORDER BY col2) AS RANK,
    col1
  FROM table_name)

SELECT col1
FROM ranking
WHERE RANK = place1;
```



# Window Frames

Now, we're finally going to define the windows of window functions in detail.

Let's start by introducing today's tables. We're going to work for a company called ColdLikeHell which deals with... well, everything that is somehow connected to ice. 

Let's check the products.

Here's the fiddle link for the schema: https://www.db-fiddle.com/f/83ivTQum95t768HCkdRLAU/2


# Exercise

Select all the information from the product table.

Just take a look at all those wonderful products! Each of them has an id, a name, and a third column which informs when it was introduced to the market.

# Exercise

Select all the information from stock_change table.

A single row in stock_change means that either a given number of products left the warehouse, or were delivered to the warehouse.

You can find:

the id of the change.
the product_id.
the quantity (positive values mean delivery, negative values mean the products left the warehouse).
the changed when it happened.

# Exercise

Select all the information from the single_order table.

A pretty simple table, each order has an id, was placed on a specific day and has a given total_price.

# Exercise

Select all the information from the order_position table.

Each order_position is a single position in an order (you may think of it as a position in an invoice). It has its own id, the id of the product, id of the order, and the quantity of the product.


# General syntax

In this part, we're finally going to learn about window frames.

Window frames define precisely which rows should be taken into account when computing the results and are always relative to the current row. 

In this way, we can create new kinds of queries.

For instance, you may say that for each row, 3 rows before and 3 rows after it are taken into account; or rows from the beginning of the partition until the current row. 

In a moment, you'll discover how such queries can come in handy. 

Take a look at the example window frame, where two rows before and two rows after the current row are selected:

<img src= 'https://learnsql.com/static/postgresql-window-functions-window-functions-part5-ex5.png'>

The are two kinds of window frames: those with the keyword ROWS and those with RANGE instead. The general syntax is as follows:

```
<window function> OVER (...
  ORDER BY <order_column>
  [ROWS|RANGE] <window frame extent>
  )
```
Of course, other elements might be added above (for instance, a PARTITION BY clause), which is why we put dots (...) in the brackets. 

For now, we'll focus on the meaning of ROWS and RANGE. 

We'll talk about PARTITION BY later in the course.

Let's take a look at the example:

```
SELECT
  id,
  total_price,
  SUM(total_price) OVER(
    ORDER BY placed
    ROWS UNBOUNDED PRECEDING)
FROM single_order;
```

In the above query, we sum the column total_price. 

For each row, we add the current row AND all the previously introduced rows (UNBOUNDED PRECEDING) to the sum. 

As a result, the sum will increase with each new order.


# Window frame definition

Let's jump into the brackets of OVER(...) and discuss the details. 

We'll start with ROWS, because they are a bit easier to explain than RANGE. The general syntax is as follows:

```
ROWS BETWEEN lower_bound AND upper_bound
```

You know BETWEEN already – it's used to define a range. 

So far, you've used it to define a range of values – this time, we're going to use it to define a range of rows instead. 

What are the two bounds? 

The bounds can be any of the five options:

- `UNBOUNDED PRECEDING` – the first possible row.

- `PRECEDING` – the n-th row before the current row (instead of n, write the number of your choice).

- `CURRENT ROW` – simply current row.

- `FOLLOWING` – the n-th row after the current row.

- `UNBOUNDED FOLLOWING` – the last possible row.

The lower bound must come BEFORE the upper bound. 

In other words, a construction like: `...ROWS BETWEEN CURRENT ROW AND UNBOUNDED PRECEDING` doesn't make sense and you'll get an error if you run it.


Exercise
Take a look at the example:

```
SELECT
  id,
  total_price,
  SUM(total_price) OVER(ORDER BY placed ROWS UNBOUNDED PRECEDING) AS running_total,
  SUM(total_price) OVER(ORDER BY placed ROWS between 3 PRECEDING and 3 FOLLOWING) AS sum_3_before_after
FROM single_order
ORDER BY placed;
```

The query computes:

- the total price of all orders placed so far (this kind of sum is called a running total).
- the total price of the current order, 3 preceding orders and 3 following orders.



First exercise

It's your turn to write a query!

Here is a syntax reminder:

```
SELECT
  id,
  quantity,
  changed,
  SUM(quantity) OVER(
    ORDER BY changed
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM stock_change
```

# Exercise

For each order, show its id, the placed date, and the third column which will count the number of orders up to the current order when sorted by the placed date.



```
SELECT
  id,
  placed,
  COUNT(id) OVER(
    ORDER BY placed
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM single_order;
```

# Exercise

Warehouse workers always need to pick the products for orders by hand and one by one. 

For positions with order_id = 5, calculate the remaining sum of all the products to pick. 

For each position from that order, show its id, the id of the product, the quantity and the quantity of the remaining items (including the current row) when sorted by the id in the ascending order.



```
SELECT
  id,
  product_id,
  quantity,
  SUM(quantity) OVER(
    ORDER BY id
    ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING)
FROM order_position
WHERE order_id = 5;
```

# Abbreviations

If our window frame has CURRENT ROW as one of the boundaries, we can also use some abbreviated syntax to make things easier:

- `ROWS UNBOUNDED PRECEDING` means `BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`
- `ROWS n PRECEDING` means `BETWEEN n PRECEDING AND CURRENT ROW`
- `ROWS CURRENT ROW` means `BETWEEN CURRENT ROW AND CURRENT ROW`

The same rules applies for FOLLOWING. As a way of example, the following query:

```
SELECT
  id,
  name,
  introduced,
  COUNT(id) OVER(ORDER BY introduced ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM product;
```

Can be rewritten to:

```
SELECT
  id,
  name,
  introduced,
  COUNT(id) OVER(ORDER BY introduced ROWS UNBOUNDED PRECEDING)
FROM product;
```

# Exercise
You will now have a chance to practice abbreviations. Pick those stock changes which refer to product_id = 3. For each of them, show the id, changed date, quantity, and the running total, indicating the current stock status. Sort the rows by the changed date in the ascending order.

```
SELECT
  id,
  changed,
  quantity,
  SUM(quantity) OVER(
    ORDER BY changed
    ROWS UNBOUNDED PRECEDING)
FROM stock_change
WHERE product_id = 3;
```

# RANGE explained

It's time to look at another type of window frame: RANGE.

The difference between ROWS and RANGE is that RANGE will take into account all rows that have the same value in the column which we order by. 

This might be helpful with dates. Consider the following problem: we want to calculate the running sum from all orders sorted by date. 

We could write something like this:

```
SELECT
  id,
  placed,
  total_price,
  SUM(total_price) OVER (ORDER BY placed ROWS UNBOUNDED PRECEDING)
FROM single_order;
```

<img src='https://learnsql.com/static/postgresql-window-functions-window-functions-part5-ex13-graphic1.png'>


And it works fine. 

But our boss could say: hey, I don't really need to see how the running sum changed during single days. 

Just show the values at the end of the day; if there are multiple orders on a single day, add them together.

The above may be implemented by changing ROWS to RANGE. 

Look how we highlighted the rows which now share the same running sum because they come from the same date.

```
SELECT
  id,
  placed,
  total_price,
  SUM(total_price) OVER(ORDER BY placed RANGE UNBOUNDED PRECEDING)
FROM single_order;
```

<img src='https://learnsql.com/static/postgresql-window-functions-window-functions-part5-ex13-graphic2.png'>

# Exercise

Modify the example so that it shows the average total_price for single days for each row.




```
SELECT
  id,
  placed,
  total_price,
  AVG(total_price) OVER(ORDER BY placed RANGE CURRENT ROW)
FROM single_order;
```

# ROWS and RANGE – explanation

The difference between ROWS and RANGE is similar to the difference between the ranking functions ROW_NUMBER and RANK()

The query with ROWS sums the total_price for all rows which have their ROW_NUMBER less than or equal to the row number of the current row.

```
SELECT
  id,
  placed,
  total_price,
  ROW_NUMBER() OVER(ORDER BY placed),
  SUM(total_price) OVER(
    ORDER BY placed
    ROWS UNBOUNDED PRECEDING)
FROM single_order
```

<img src='https://learnsql.com/static/postgresql-window-functions-window-functions-part5-ex14-graphic1.png'>

The query with RANGE sums the total_price for all rows which have their RANK() less than or equal to the rank of the current row.

```
SELECT
  id,
  placed,
  total_price,
  RANK() OVER(ORDER BY placed),
  SUM(total_price) OVER(
    ORDER BY placed
    RANGE UNBOUNDED PRECEDING)
FROM single_order
```

<img src='https://learnsql.com/static/postgresql-window-functions-window-functions-part5-ex14-graphic2.png'>

# Boundaries with RANGE

The window frame of RANGE is defined just like the window frame of ROWS: with BETWEEN ... AND ... or the abbreviated version.

You can use RANGE UNBOUNDED PRECEDING and RANGE BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING, as well as RANGE CURRENT ROW. 


# Exercise

For each stock_change with product_id = 7, show its id, quantity, changed date and another column to count the number of stock changes with product_id = 7 on that particular date.

```
SELECT
  id,
  quantity,
  changed,
  COUNT(id) OVER(ORDER BY changed RANGE CURRENT ROW)
FROM stock_change
WHERE product_id = 7;
```

# Exercise

Our finance department needs to calculate future cashflows for each date. 

Let's help them. 

In order to do that, we need to show each order: its id, placed date, total_price and the total sum of all prices of orders from the very same day or any later date.

```
SELECT
  id,
  placed,
  total_price,
  SUM(total_price) OVER(
    ORDER BY placed
    RANGE BETWEEN CURRENT ROW
      AND UNBOUNDED FOLLOWING)
FROM single_order;
```


# Default window frame – without ORDER BY

You may wonder what the default window frame is when it's not explicitly specified. This may differ between databases, but the most typical rule is as follows:

If you don't specify an ORDER BY clause within OVER(...), the whole partition of rows will be used as the window frame.

If you do specify an ORDER BY clause within OVER(...), the database will assume RANGE UNBOUNDED PRECEDING as the window frame. Let's check both of these cases in exercises.

# Exercise

We'll start with not specifying an ORDER BY clause within OVER(...).

For each single order, show its id, date when it was placed, the total price and the sum of all total prices.

Note that the SUM computes the sum of all prices in the table, even though you did not specify the window frame.

```
SELECT
  id,
  placed,
  total_price,
  SUM(total_price) OVER()
FROM single_order;
```

# Default window frame – with ORDER BY

When there is no ORDER BY clause in OVER(...), the query simply treats all rows as the window frame for each row. 

Nothing shocking, really – that's the kind of queries we wrote in previous parts.

Now, we said the following: if there is an ORDER BY clause, RANGE UNBOUNDED PRECEDING will be used as the default window frame. 

Let's find out if it's true.

# Exercise

Just as in one of the previous exercises, we'll be looking for the running sum of single orders. 

For each order, show its id, placed date, total_price and the sum of all total prices. 

Sort the orders by the placed date, but do not specify any window frame.

The sum of total_prices should be calculated as if you wrote RANGE UNBOUNDED PRECEDING.

```
SELECT
  id,
  placed,
  total_price,
  SUM(total_price) OVER(ORDER BY placed)
FROM single_order;
```

# Summary

Alright! It's time to review what we've learned in this part:

- You can define a window frame within OVER(...). The syntax is: `[ROWS|RANGE] <window frame definition>.`

- `ROWS` always treats rows individually (like the ROW_NUMBER() function), RANGE also adds rows which share the same value in the column we order by (like the RANK() function).

- `<window frame definition>` is defined with BETWEEN <lower bound> AND <upper bound>, where the bounds may be defined with:
    - UNBOUNDED PRECEDING,
    - n PRECEDING (ROWS only),
    - CURRENT ROW,
    - n FOLLOWING (ROWS only),
    - UNBOUNDED FOLLOWING

# HOMEWORK

The fiddle link for the homework is: https://www.db-fiddle.com/f/3t1nZ8AQBJdVpApcJqJLGn/0

Use the application table for the next two exercises.


# Exercise

For each application, show its name, average_rating and its rank, with best rated apps coming first.

```
SELECT
  name,
  average_rating,
  RANK() OVER (ORDER BY average_rating DESC)
FROM application;
```

# Exercise

Find the application that ranked 3rd in terms of the greatest number of downloads. Show its name and the number of downloads.

```
WITH ranking AS (
  SELECT
    name,
    downloads,
    RANK() OVER(ORDER BY downloads DESC) AS rank
  FROM application
)

SELECT
  name,
  downloads
FROM ranking
WHERE rank = 3;
```

# For the next set of questions, use the schema from the Window Frame section of the class: https://www.db-fiddle.com/f/83ivTQum95t768HCkdRLAU/2

# Exercise

For each product, show its id, name, introduced date and the count of products introduced up to that point.

```
SELECT
  id,
  name,
  introduced,
  COUNT(id) OVER(
    ORDER BY introduced
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM product;
```

# Exercise

Now, for each single_order, show its placed date, total_price, the average price calculated by taking 2 previous orders, the current order and 2 following orders (in terms of the placed date) and the ratio of the total_price to the average price calculated as before.

```
SELECT
  placed,
  total_price,
  AVG(total_price) OVER(ORDER BY placed ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING),
  total_price / AVG(total_price) OVER(ORDER BY placed ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING)
FROM single_order;
```

# Exercise

For each single_order, show its placed date, total_price and the average price from the current single_order and three previous orders (in terms of the placed date).

```
SELECT
  placed,
  total_price,
  AVG(total_price) OVER(ORDER BY placed ROWS BETWEEN 3 PRECEDING AND CURRENT ROW)
FROM single_order;
```

# Exercise

For each stock_change, show id, product_id, quantity, changed date, and the total quantity change from all stock_change for that product.

```
SELECT
  id,
  product_id,
  quantity,
  changed,
  SUM(quantity) OVER(ORDER BY product_id RANGE CURRENT ROW)
FROM stock_change;
```

# Exercise

For each stock_change, show its id, changed date, and the number of any stock changes that took place on the same day or any time earlier.

```
SELECT
  id,
  changed,
  COUNT(id) OVER(ORDER BY changed RANGE UNBOUNDED PRECEDING)
FROM stock_change;
```